In [2]:
# EDIT > NOTEBOOK SETTINGS > GPU

!pip3 install -q tensorflow_gpu>=2.0

import tensorflow as tf
print(tf.__version__)


ERROR: tensorflow 2.2.0rc1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc1 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
2.1.0


In [4]:
# verify GPU availability
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
else:
  print ("Success")

Success


In [5]:
!pip3 install -q ktrain

print('ktrain installed')

     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 6.7MB 56.4MB/s 
     |████████████████████████████████| 10.0MB 49.1MB/s 
     |████████████████████████████████| 983kB 54.7MB/s 
     |████████████████████████████████| 245kB 55.5MB/s 
     |████████████████████████████████| 1.8MB 57.1MB/s 
     |████████████████████████████████| 542kB 55.3MB/s 
     |████████████████████████████████| 1.0MB 47.5MB/s 
     |████████████████████████████████| 3.7MB 62.3MB/s 
     |████████████████████████████████| 870kB 65.4MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=0.25.0; python_version >= "3.0", but you'll have pandas 1.0.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [6]:
# using code from second tutorial (distilbert tutorial)

categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [16]:
# THIS CELL DOES NOT RUN -- TUTORIAL CODE IS BROKEN
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=train_b.target_names)

print(ktrain.__version__)
print(keras.__version__)

#errors with preprocess_train
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:366: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


ERROR! Session/line number was not unique in database. History logging moved to new session 59
0.11.3
2.2.5
preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


KeyError: ignored

In [17]:
# SWITCHING TO USING CODE FROM FIRST TUTORIAL
# this means IMDB data (rip sean)

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

ERROR! Session/line number was not unique in database. History logging moved to new session 60
84131840/84125825 [==============================] - 9s 0us/step


In [18]:
# set path to dataset
import os.path
dataset = '/root/.keras/datasets/aclImdb'
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(IMDB_DATADIR)

/root/.keras/datasets/aclImdb


In [25]:
# code does not work -- same error as above
# error with distilbert?

# update model name for desired transformer
MODEL_NAME = 'distilbert' # also could user BERT

(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='distilbert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


KeyError: ignored

In [24]:
model = text.text_classifier(MODEL_NAME, (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

ValueError: ignored

In [23]:
# does not work with BERT (runs out of memory)

learner.fit_onecycle(2e-5, 1)
text.text_classifier()



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
    6/25000 [..............................] - ETA: 53:43:56

ResourceExhaustedError: ignored

In [0]:
# taking code from second tutorial again

In [0]:
# CODE FROM OTHER TUTORIAL BELOW

# installs 
!pip install pytorch-pretrained-bert pytorch-nlp

In [0]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)